In [1]:
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl
#!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl
#!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import Adam
import torch.nn.functional as F
from transformers import EarlyStoppingCallback
import torch

import torch
import transformers
import sklearn
import os

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from transformers import DistilBertTokenizerFast, AutoModelForSequenceClassification, Trainer
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from accelerate import cpu_offload, dispatch_model
from accelerate.utils.modeling import infer_auto_device_map
from torch.utils.data import Dataset, DataLoader
from peft import LoraConfig, TaskType
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
import pandas as pd
import numpy as np
import accelerate
import transformers
from transformers import (

    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,

    Trainer,
    TrainingArguments
)

tqdm.pandas()

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class Dataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
        )
        return {
            'input_ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
        }

In [4]:
from peft import PeftModel, PeftConfig

# Define the ensembling weights for each model
model_weights = [0.1, 0.05, 0, 0.99, 0]
test_data = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
X_test = list(test_data["text"])


tokenizers = [
    AutoTokenizer.from_pretrained('/kaggle/input/dis-lora/dis_lora_token/content/token'),
    AutoTokenizer.from_pretrained('/kaggle/input/roberta-loraa/token2/content/token'),
    AutoTokenizer.from_pretrained('/kaggle/input/unbert-loraa/token_unbert_lora/content/token'),
    AutoTokenizer.from_pretrained('/kaggle/input/albertlora-v3/tokenizer/content/tokenizer_albert'),
    AutoTokenizer.from_pretrained('/kaggle/input/xlnet-lora/tokenizer_xlnet/content/tokenizer_xlnet')
]

model = []

foundation_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/transformers/distilbert-base-uncased',num_labels=1, )
model1 = PeftModel.from_pretrained(foundation_model, '/kaggle/input/dis-lora/dis_lora_model/content/model', is_trainable=False).to('cuda')
model.append(model1)


foundation_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/transformers/roberta-base',num_labels=1, )
model2 = PeftModel.from_pretrained(foundation_model, '/kaggle/input/roberta-loraa/model2/content/model', is_trainable=False).to('cuda')
model.append(model2)


foundation_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/transformers/bert-base-uncased',num_labels=1, )
model3 = PeftModel.from_pretrained(foundation_model, '/kaggle/input/unbert-loraa/model_unbert_lora/content/model', is_trainable=False).to('cuda')
model.append(model3)


foundation_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/huggingface-albert-v2/albert-base-v2',num_labels=1, )
model4 = PeftModel.from_pretrained(foundation_model, '/kaggle/input/albertlora-v3/model_3/content/albert', is_trainable=False).to('cuda')
model.append(model4)


foundation_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/xlnet-1/file_4/content/output/checkpoint-2500',num_labels=1, )
model5 = PeftModel.from_pretrained(foundation_model, '/kaggle/input/xlnet-lora/xlnet/content/xlnet', is_trainable=False).to('cuda')
model.append(model5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/transformers/distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/transformers/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/transformers/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRA

In [5]:
# Generate the predictions on the test set using a weighted ensemble
predictions = torch.zeros(len(test_data), 1, device='cuda')
for i, model in enumerate(model):
    tokenizer = tokenizers[i]
    test_dataset = Dataset(X_test, tokenizer, max_len=512)
    test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

    model.eval()
    with torch.no_grad():
        for j, batch in enumerate(test_loader):
            input_ids = batch['input_ids'].to('cuda')
            attention_mask = batch['attention_mask'].to('cuda')

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            predictions[j*8:(j+1)*8, :] += model_weights[i] * logits

In [6]:
test_pred = (torch.tensor(predictions)).cpu().numpy().ravel()

/tmp/ipykernel_26/2822908032.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_pred = (torch.tensor(predictions)).cpu().numpy().ravel()


In [7]:
submission = pd.DataFrame.from_dict({
    'id': test_data['id'],
    'prediction': test_pred
})
submission.to_csv('submission.csv', index=False)

In [8]:
submission.head()

,id,prediction
0,0000aaaa,0.139474
1,1111bbbb,0.080989
2,2222cccc,-0.012128
